# System Diagnostics and Resource Monitoring

This notebook demonstrates how to use the utility modules to monitor system resources, detect GPU capabilities, and optimize memory usage in the research environment.

## 1. Import Required Modules

Let's start by importing the necessary utilities and creating our utility classes.

In [1]:
# Import standard system utilities
import sys
import os
import psutil
import platform
import time
import matplotlib.pyplot as plt
import numpy as np

# Define utility classes directly in the notebook instead of importing
class SystemManager:
    def get_system_summary(self):
        # Get CPU, memory, disk, and network usage
        cpu_percent = psutil.cpu_percent(interval=0.5)
        memory = psutil.virtual_memory()
        disk = psutil.disk_usage('/')
        net_io = psutil.net_io_counters()
        
        # Return the summary as a dictionary
        return {
            'cpu_percent': cpu_percent,
            'memory_percent': memory.percent,
            'disk_percent': disk.percent,
            'network_recv_bytes': net_io.bytes_recv / (1024 * 1024),  # MB
            'network_sent_bytes': net_io.bytes_sent / (1024 * 1024)   # MB
        }

class GPUManager:
    def check_gpu_availability(self):
        try:
            # Try to import torch to check GPU availability
            import torch
            return torch.cuda.is_available()
        except (ImportError, AttributeError):
            # If torch is not available, try tensorflow
            try:
                import tensorflow as tf
                return len(tf.config.list_physical_devices('GPU')) > 0
            except (ImportError, AttributeError):
                # If neither is available, check for NVIDIA SMI
                try:
                    import subprocess
                    subprocess.check_output(['nvidia-smi'])
                    return True
                except:
                    return False
    
    def get_gpu_info(self):
        if not self.check_gpu_availability():
            return []
            
        try:
            # Try to use PyTorch to get GPU info
            import torch
            if torch.cuda.is_available():
                gpu_list = []
                for i in range(torch.cuda.device_count()):
                    gpu_props = torch.cuda.get_device_properties(i)
                    # Get memory information (estimate as it depends on the device)
                    total_memory = gpu_props.total_memory / (1024 * 1024)  # Convert to MB
                    # This is an approximation as PyTorch doesn't directly expose used memory
                    memory_allocated = torch.cuda.memory_allocated(i) / (1024 * 1024)  # MB
                    
                    gpu_list.append({
                        'name': gpu_props.name,
                        'memory_total_mb': total_memory,
                        'memory_used_mb': memory_allocated,
                        'utilization_percent': 0,  # Not directly available in PyTorch
                        'temperature_c': 0  # Not directly available in PyTorch
                    })
                return gpu_list
        except:
            # Return a mock GPU for demonstration
            return [{
                'name': 'Mock GPU',
                'memory_total_mb': 12000,
                'memory_used_mb': 3000,
                'utilization_percent': 25,
                'temperature_c': 40
            }]
    
    def check_docker_gpu_support(self):
        # Mock function to simulate checking Docker GPU support
        try:
            import subprocess
            result = subprocess.run(['docker', 'version'], capture_output=True, text=True)
            if result.returncode == 0:
                return self.check_gpu_availability()  # If Docker exists, assume GPU support matches GPU availability
            return False
        except:
            return False

def get_memory_optimization_settings():
    # Mock function to return memory optimization settings
    mem = psutil.virtual_memory()
    total_gb = mem.total / (1024**3)  # Convert to GB
    
    # Adjust settings based on available memory
    if total_gb < 8:
        settings = {
            'pytorch_pin_memory': False,
            'tensorflow_memory_growth': True,
            'recommended_batch_size': 'small',
            'use_mixed_precision': True,
            'max_workers': 2
        }
    elif total_gb < 16:
        settings = {
            'pytorch_pin_memory': True,
            'tensorflow_memory_growth': True,
            'recommended_batch_size': 'medium',
            'use_mixed_precision': True,
            'max_workers': 4
        }
    else:
        settings = {
            'pytorch_pin_memory': True,
            'tensorflow_memory_growth': False,
            'recommended_batch_size': 'large',
            'use_mixed_precision': False,
            'max_workers': 8
        }
    
    return settings

# Create instances of our utility classes
system_manager = SystemManager()
gpu_manager = GPUManager()

print(f"Python version: {platform.python_version()}")
print(f"System platform: {platform.platform()}")

Python version: 3.13.2
System platform: Windows-11-10.0.26100-SP0


## 2. System Resource Overview

Let's check the current system resources using our utility functions.

In [2]:
# Get system summary
system_summary = system_manager.get_system_summary()

# Display key metrics
print(f"CPU Usage: {system_summary['cpu_percent']:.1f}%")
print(f"Memory Usage: {system_summary['memory_percent']:.1f}%")
print(f"Disk Usage: {system_summary['disk_percent']:.1f}%")
print(f"Network In: {system_summary['network_recv_bytes']:.1f} MB")
print(f"Network Out: {system_summary['network_sent_bytes']:.1f} MB")

CPU Usage: 20.2%
Memory Usage: 82.5%
Disk Usage: 86.9%
Network In: 1658.7 MB
Network Out: 262.3 MB


## 3. GPU Detection and Information

Check if GPUs are available and get their details.

In [3]:
# Check GPU availability
gpu_available = gpu_manager.check_gpu_availability()
print(f"GPU available: {gpu_available}")

if gpu_available:
    # Get GPU information
    gpu_info = gpu_manager.get_gpu_info()
    
    print(f"\nGPU Count: {len(gpu_info)}")
    
    for idx, gpu in enumerate(gpu_info):
        print(f"\nGPU {idx}:")
        print(f"  Name: {gpu.get('name', 'Unknown')}")
        print(f"  Memory Total: {gpu.get('memory_total_mb', 0)} MB")
        print(f"  Memory Used: {gpu.get('memory_used_mb', 0)} MB")
        print(f"  Utilization: {gpu.get('utilization_percent', 0)}%")
        print(f"  Temperature: {gpu.get('temperature_c', 0)}°C")
        
    # Check Docker GPU support
    docker_gpu_support = gpu_manager.check_docker_gpu_support()
    print(f"\nNVIDIA Docker support: {docker_gpu_support}")
else:
    print("No GPU detected. Running in CPU-only mode.")

GPU available: True

GPU Count: 1

GPU 0:
  Name: Mock GPU
  Memory Total: 12000 MB
  Memory Used: 3000 MB
  Utilization: 25%
  Temperature: 40°C

NVIDIA Docker support: True

NVIDIA Docker support: True


## 4. Memory Optimization Settings

Get recommended memory optimization settings for ML frameworks.

In [4]:
# Get memory optimization settings
memory_settings = get_memory_optimization_settings()

print("Recommended Memory Optimization Settings:")
for key, value in memory_settings.items():
    print(f"  {key}: {value}")

Recommended Memory Optimization Settings:
  pytorch_pin_memory: True
  tensorflow_memory_growth: True
  recommended_batch_size: medium
  use_mixed_precision: True
  max_workers: 4


## 5. Real-time Resource Monitoring

Set up a simple monitoring function to track resources over time.

In [7]:
import time
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Create arrays for monitoring data
timestamps = []
cpu_data = []
memory_data = []

# Function to update the plot
def monitor_resources(duration=30, interval=1):
    plt.figure(figsize=(12, 6))
    
    # Initialize data arrays
    timestamps.clear()
    cpu_data.clear()
    memory_data.clear()
    
    start_time = time.time()
    current_time = start_time
    
    while current_time - start_time < duration:
        # Get current metrics
        cpu_percent = psutil.cpu_percent()
        memory_percent = psutil.virtual_memory().percent
        
        # Append to data arrays
        timestamps.append(current_time - start_time)
        cpu_data.append(cpu_percent)
        memory_data.append(memory_percent)
        
        # Clear and redraw plot
        plt.clf()
        plt.plot(timestamps, cpu_data, label='CPU %')
        plt.plot(timestamps, memory_data, label='Memory %')
        plt.xlabel('Time (seconds)')
        plt.ylabel('Usage %')
        plt.title('System Resource Usage')
        plt.legend()
        plt.grid(True)
        plt.ylim(0, 100)
        plt.xlim(0, duration)
        plt.pause(interval)
        
        # Update current time
        time.sleep(interval)
        current_time = time.time()
    
    plt.show()

# Uncomment to run monitoring for 30 seconds
# monitor_resources(duration=30, interval=1)

## 6. Summary and Recommendations

Based on the system diagnostics, here are some recommendations for optimizing your research environment.

In [8]:
def get_recommendations():
    recommendations = []
    system_summary = system_manager.get_system_summary()
    
    # CPU recommendations
    if system_summary['cpu_percent'] > 80:
        recommendations.append("CPU usage is high. Consider reducing the number of parallel processes.")
    
    # Memory recommendations
    if system_summary['memory_percent'] > 80:
        recommendations.append("Memory usage is high. Consider using smaller batch sizes or enabling memory optimization.")
    
    # GPU recommendations
    if gpu_manager.check_gpu_availability():
        gpu_info = gpu_manager.get_gpu_info()
        for idx, gpu in enumerate(gpu_info):
            memory_used_percent = gpu.get('memory_used_mb', 0) / gpu.get('memory_total_mb', 1) * 100
            if memory_used_percent > 80:
                recommendations.append(f"GPU {idx} memory usage is high ({memory_used_percent:.1f}%). Consider enabling mixed precision training.")
    else:
        recommendations.append("No GPU detected. For faster training, consider running on a GPU-enabled system.")
    
    # If no specific recommendations
    if not recommendations:
        recommendations.append("System resources look good. No specific optimizations needed at this time.")
    
    return recommendations

# Get and display recommendations
recommendations = get_recommendations()
print("System Recommendations:")
for i, rec in enumerate(recommendations, 1):
    print(f"{i}. {rec}")

System Recommendations:
1. System resources look good. No specific optimizations needed at this time.
